In [1]:
import pickle
import numpy as np
import os

def load_cifar10_batch(batch_file):
    """Load a single batch of CIFAR-10."""
    with open(batch_file, 'rb') as f:
        batch = pickle.load(f, encoding='latin1')
        data = batch['data']
        labels = batch['labels']
        data = data.reshape((len(data), 3, 32, 32))
        data = data.transpose(0, 2, 3, 1)  # Chuyển từ (N, 3, 32, 32) sang (N, 32, 32, 3)
    return data, labels

def load_cifar10(data_dir):
    """Load all of CIFAR-10."""
    train_data = []
    train_labels = []
    for i in range(1, 6):
        batch_file = os.path.join(data_dir, 'data_batch_{}'.format(i))
        data, labels = load_cifar10_batch(batch_file)
        train_data.append(data)
        train_labels.append(labels)

    train_data = np.concatenate(train_data)
    train_labels = np.concatenate(train_labels)

    test_data, test_labels = load_cifar10_batch(os.path.join(data_dir, 'test_batch'))

    return train_data, train_labels, test_data, test_labels

# Đường dẫn tới thư mục chứa các tệp CIFAR-10 đã giải nén
data_dir = 'cifar-10-python\cifar-10-batches-py'

# Đọc dữ liệu
train_data, train_labels, test_data, test_labels = load_cifar10(data_dir)

test_labels = np.array(test_labels)

print('Train data shape:', train_data.shape)
print('Train labels shape:', train_labels.shape)
print('Test data shape:', test_data.shape)
print('Test labels shape:', test_labels.shape)


Train data shape: (50000, 32, 32, 3)
Train labels shape: (50000,)
Test data shape: (10000, 32, 32, 3)
Test labels shape: (10000,)


In [2]:
num_training = 47000
num_validation = 3000



mask = range(num_training, num_training + num_validation)
Val_data = train_data[mask]
Val_labels = train_labels[mask]

mask = range(num_training)
Train_data = train_data[mask]
Train_labels = train_labels[mask]

In [3]:
import cv2

def compute_hog_features(images):
    hog_features_list = []
    hog_descriptor = cv2.HOGDescriptor(
        _winSize=(32 // 8 * 8, 32 // 8 * 8),
        _blockSize=(16, 16),
        _blockStride=(8, 8),
        _cellSize=(8, 8),
        _nbins=9
    )

    for image in images:
        # Chuyển đổi ảnh RGB sang grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        # Tính toán HOG đặc trưng
        hog_features = hog_descriptor.compute(gray_image)
        hog_features_list.append(hog_features.flatten())
    
    hog_features_array = np.array(hog_features_list)
    return hog_features_array

In [4]:
Train_data = compute_hog_features(Train_data)
Val_data = compute_hog_features(Val_data)
test_data = compute_hog_features(test_data)

In [5]:
X_train = Train_data
y_train = Train_labels
X_val = Val_data
y_val = Val_labels
X_test = test_data
y_test = test_labels

In [6]:
from sklearn import svm   #After running various iterations, the polynomial kernel was chosen as the best kernel in this case.
svc = svm.SVC(probability=False,  kernel="poly", C=0.01)
svc.fit(X_train, y_train)

SVC(C=0.01, kernel='poly')

In [7]:
pred = svc.predict(X_train)
pred = np.array(pred)
y_train = np.array(y_train)
acc_train = np.mean(pred == y_train)
print('Train Accuracy = {0:f}'.format(acc_train))

Train Accuracy = 0.635021


In [8]:
pred = svc.predict(X_test)
pred = np.array(pred)
y_test = np.array(y_test)
acc_test = np.mean(pred == y_test)
print('Test Accuracy = {0:f}'.format(acc_test))

Test Accuracy = 0.593500


Lỏ